<a href="https://colab.research.google.com/github/skovz99/2D-and-3D-colocalization/blob/main/2D_and_3D_Total_Colocalization_Image_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import skimage
import skimage.io
import skimage.util as util
from skimage import filters, measure
import math

Image1 = skimage.io.imread('')
Image2 = skimage.io.imread('')

def total_colocalization_2D(image_1, image_2, channel_image_1, channel_image_2, threshold = 50):
    if channel_image_1 == 'red':
      channel_1 = image_1[:, :, 0]
    elif channel_image_1 == 'green':
      channel_1 = image_1[:, :, 1]
    elif channel_image_1 == 'blue':
      channel_1 = image_1[:, :, 2]
    if channel_image_2 == 'red':
      channel_2 = image_2[:, :, 0]
    elif channel_image_2 == 'green':
      channel_2 = image_2[:, :, 1]
    elif channel_image_2 == 'blue':
      channel_2 = image_2[:, :, 2]
    thresholded_image_1 = (channel_1 > threshold).astype(int)
    thresholded_image_2 = (channel_2 > threshold).astype(int)
    image_1_df = pd.DataFrame(thresholded_image_1)
    image_2_df = pd.DataFrame(thresholded_image_2)
    add = image_1_df + image_2_df
    numpy_add = add.to_numpy()
    overlap_counts = np.count_nonzero(numpy_add == 2)
    numpy_image_1 = image_1_df.to_numpy()
    numpy_image_2 = image_2_df.to_numpy()
    total_image_1_df = np.count_nonzero(numpy_image_1 == 1)
    total_image_2_df = np.count_nonzero(numpy_image_2 == 1)
    colocalization_image_1 = round(((overlap_counts)/(total_image_1_df)) * 100, 2)
    colocalization_image_2 = round(((overlap_counts)/(total_image_2_df)) * 100, 2)
    return colocalization_image_1, colocalization_image_2

add1a, add2a = total_colocalization_2D(Image1, Image2, channel_image_1 = 'blue', channel_image_2 = 'red', threshold = 50)
add1a, add2a

(19.45, 6.98)

In [ ]:
import pandas as pd
import numpy as np
import skimage
import skimage.io
import skimage.util as util
from skimage import filters, measure
import math
import matplotlib.pyplot as plt

def total_colocalization_3D(image_1, image_2, channel_image_1, channel_image_2, threshold = 50, pixel_intensity_similarity_gradient = 'True', pixel_intensity_similarity_int=10):
  if channel_image_1 == 'red':
    channel_1 = image_1[:, :, 0]
  elif channel_image_1 == 'green':
    channel_1 = image_1[:, :, 1]
  elif channel_image_1 == 'blue':
    channel_1 = image_1[:, :, 2]
  if channel_image_2 == 'red':
    channel_2 = image_2[:, :, 0]
  elif channel_image_2 == 'green':
    channel_2 = image_2[:, :, 1]
  elif channel_image_2 == 'blue':
    channel_2 = image_2[:, :, 2]
  image_1_df = pd.DataFrame(channel_1)
  image_2_df = pd.DataFrame(channel_2)
  thresholded_image_1 = (channel_1 > threshold).astype(int)
  thresholded_image_2 = (channel_2 > threshold).astype(int)
  image_1_dfa = pd.DataFrame(thresholded_image_1)
  image_2_dfa = pd.DataFrame(thresholded_image_2)
  add = image_1_dfa + image_2_dfa
  numpy_image_1 = image_1_dfa.to_numpy()
  numpy_image_2 = image_2_dfa.to_numpy()
  total_image_1_df = np.count_nonzero(numpy_image_1 == 1)
  total_image_2_df = np.count_nonzero(numpy_image_2 == 1)
  # the add dataframe adds the thresholded (0 = background, 1 = foreground) together to get a mix of 0, 1 and 2's. everything less than 2 needs to be converted to zero
  # since both 0 and 1 mean that there is no overlap, have to get rid of it by converting both to zero
  add[add <= 1] = 0
  add[add == 2] = 1
  # multiply the adjusted 'add' df by the df with the unthresholded first image to return all the pixel intensities where there is overlap between image 1 and image 2 onto the non-thresholded image 1
  multiply_img_1 = (image_1_df) * add
  # do the same to the non-thresholded image 2
  multiply_img_2 = (image_2_df) * add
  # subtract the multiplied images from each other
  subtract = multiply_img_1 - multiply_img_2
  subtract = abs(subtract)
  if pixel_intensity_similarity_gradient == 'True':
    subtract_normalized = subtract / 255
  else:
    subtract_normalized = np.where(subtract > pixel_intensity_similarity_int, 0, 1)
  sum_subtract_normalized = subtract_normalized[subtract_normalized != 0].sum().sum()
  # calculate the percentage based on the 2's (now 1's) of overlap in the thresholded image combinations
  total_3D_sum_img_1 = round(((sum_subtract_normalized)/(total_image_1_df) * 100), 2)
  total_3D_sum_img_2 = round(((sum_subtract_normalized)/(total_image_2_df) * 100), 2)
  return total_3D_sum_img_1, total_3D_sum_img_2

Image1 = skimage.io.imread('')
Image2 = skimage.io.imread('')

add1, add2 = total_colocalization_3D(Image1, Image2, channel_image_1 = 'blue', channel_image_2 = 'red', pixel_intensity_similarity_gradient = 'True', pixel_intensity_similarity_int=10)
add1, add2


(5.36, 1.92)

In [ ]:
# define a function to calculate the difference between 2D total colocalization and 3D total colocalization
def difference_2D_3D_total(twoD_img_1_overlap_percent, twoD_img_2_overlap_percent, threeD_img_1_overlap_percent, threeD_img_2_overlap_percent, as_percent_of_2D = 'False'):
  # calculate the difference between 2D and 3D when image 1 was used as the denominator
  if as_percent_of_2D == 'False':
    change_from_2D_img_1 = round((threeD_img_1_overlap_percent) - (twoD_img_1_overlap_percent),2)
    change_from_2D_img_2 = round((threeD_img_2_overlap_percent) - (twoD_img_2_overlap_percent),2)
  else:
    change_from_2D_img_1 = round((((threeD_img_1_overlap_percent) - (twoD_img_1_overlap_percent))/(twoD_img_1_overlap_percent)) * 100,2)
    change_from_2D_img_2 = round((((threeD_img_2_overlap_percent) - (twoD_img_2_overlap_percent))/(twoD_img_2_overlap_percent)) * 100,2)
  return change_from_2D_img_1, change_from_2D_img_2

diff = difference_2D_3D_total(add1a, add2a, add1, add2, as_percent_of_2D = 'False')
diff

(-14.09, -5.06)